In [2]:
# import necessary packages
import pandas as pd 
import numpy as np
import os
import shutil
from datetime import datetime
from datetime import date
from datetime import timedelta
import tushare as ts

In [122]:
# in this block, I merge all the data to a single file

base_dir = "/Users/alan/Desktop/BI/Thesis/1st_draft/Data/StockTwits"

df = pd.DataFrame(columns=["body","sentiment"])

# count the amount of files contained in all the folders
count = 0
for folder_name in os.listdir(base_dir):
    if not folder_name.startswith(".") and not folder_name.endswith(".csv"):
        folder_path = os.path.join(base_dir,folder_name)
        ## Here, I just want to concatenate 2 columns which are "body" and "sentiment"
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path,file_name)
            df_sub = pd.read_csv(file_path)
            for row in df_sub.iterrows():
                count += 1

checkpoints = {int(count*perc/100): perc for perc in range(0,100,1)}

counter = 0
for folder_name in os.listdir(base_dir):
    if not folder_name.startswith(".") and not folder_name.endswith(".csv"):
        folder_path = os.path.join(base_dir,folder_name)
        ## Here, I just want to concatenate 2 columns which are "body" and "sentiment"
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path,file_name)
            df_sub = pd.read_csv(file_path)
            for row in df_sub.iterrows():
                if ast.literal_eval(row[1]["entities"])["sentiment"] == None:
                    counter += 1
                    ## I also add the check point so I know the progress
                    if counter+1 in checkpoints:
                        print("{}% of the rows have been processed at {}".format(checkpoints[counter+1], datetime.utcnow()))
                    continue
                else:
                    counter += 1
                    body = row[1]["body"]
                    sentiment = ast.literal_eval(row[1]["entities"])["sentiment"]["basic"]
                    data = np.array([[body,sentiment]])
                    row_new = pd.DataFrame(data, columns=["body","sentiment"])
                    df = pd.concat([df, row_new], ignore_index=True)
                
                if counter+1 in checkpoints:
                    print("{}% of the rows have been processed at {}".format(checkpoints[counter+1], datetime.utcnow()))

1% of the rows have been processed at 2023-10-26 14:17:10.462358
2% of the rows have been processed at 2023-10-26 14:17:18.070459
3% of the rows have been processed at 2023-10-26 14:17:33.701373
4% of the rows have been processed at 2023-10-26 14:18:04.478746
5% of the rows have been processed at 2023-10-26 14:18:44.025202
6% of the rows have been processed at 2023-10-26 14:19:40.543353
7% of the rows have been processed at 2023-10-26 14:20:50.341588
8% of the rows have been processed at 2023-10-26 14:22:09.278401
9% of the rows have been processed at 2023-10-26 14:24:03.488094
10% of the rows have been processed at 2023-10-26 14:26:19.018052
11% of the rows have been processed at 2023-10-26 14:28:35.988304
12% of the rows have been processed at 2023-10-26 14:30:56.161966
13% of the rows have been processed at 2023-10-26 14:33:44.860202
14% of the rows have been processed at 2023-10-26 14:36:49.727401
15% of the rows have been processed at 2023-10-26 14:40:37.847612
16% of the rows hav

In [5]:
# now, we merge the data with the indian stock market data
df = pd.read_csv("/Users/alan/Desktop/BI/Thesis/1st_draft/Data/merged_data.csv")
df_india = pd.read_csv("/Users/alan/Desktop/BI/Thesis/1st_draft/Data/StockTwits/tweets.csv").iloc[:,1:]
df_india.columns = df.columns
df_merged = pd.concat([df,df_india],ignore_index=True)
df_merged = pd.concat([df,df_india],ignore_index=True)
df_merged.drop_duplicates(inplace=True)

,body,sentiment
count,2090191,2090191
unique,2084953,2
top,$TSLA its time,Bullish
freq,2,1570211


In [9]:
df_merged.to_csv("fine_tune_dataset.csv")

In [37]:
# In this chunk, I will try to collect the stock market data from Tushare
ts.set_token("82ae03fc12b920fc793249158a6956785a6093caf0fdbae2c4982cd0")
pro = ts.pro_api()

df_SH = pro.index_daily(ts_code = "000001.SH")
df_SH.to_csv("/Users/alan/Desktop/BI/Thesis/1st_draft/Data/SH.csv",index = False)
df_SZ = pro.index_daily(ts_code = "399005.SZ")
df_SZ.to_csv("/Users/alan/Desktop/BI/Thesis/1st_draft/Data/SZ.csv",index = False)

In [105]:
# in this chunk, I extract all the available information of the script of CCTV
start_date = date(2006,7,1)
end_date = date(2023,10,26)
current_date = start_date

month = 0
df_cctvnews = pd.DataFrame(columns = ["date","title","content"])
while current_date <= end_date:
    string = str(current_date)
    date_str = string[:4]+string[5:7]+string[8:] 
    df_news = pro.cctv_news(date = date_str)
    df_cctvnews = pd.concat([df_cctvnews,df_news],ignore_index=True)
    current_date += timedelta(days = 1)
    month += 1
    if month % 30 == 0:
        print("Now we are already on {}.".format(date_str))
        
df_cctvnews.to_csv("/Users/alan/Desktop/BI/Thesis/1st_draft/Data/cctvnews.csv",index=False)

Now we are already on 20160417.
Now we are already on 20160517.
Now we are already on 20160616.
Now we are already on 20160716.
Now we are already on 20160815.
Now we are already on 20160914.
Now we are already on 20161014.
Now we are already on 20161113.
Now we are already on 20161213.
Now we are already on 20170112.
Now we are already on 20170211.
Now we are already on 20170313.
Now we are already on 20170412.
Now we are already on 20170512.
Now we are already on 20170611.
Now we are already on 20170711.
Now we are already on 20170810.
Now we are already on 20170909.
Now we are already on 20171009.
Now we are already on 20171108.
Now we are already on 20171208.
Now we are already on 20180107.
Now we are already on 20180206.
Now we are already on 20180308.
Now we are already on 20180407.
Now we are already on 20180507.
Now we are already on 20180606.
Now we are already on 20180706.
Now we are already on 20180805.
Now we are already on 20180904.
Now we are already on 20181004.
Now we a

In [116]:
# Now I ll try the media news
df = pro.news(src='eastmoney', start_date="2019-01-01", end_date="2019-03-01")
df

,datetime,content,title
0,2019-02-28 23:57:14,【阿联酋修建巨型地下战略油库】阿联酋阿布扎比国家石油公司日前宣布，该公司正在阿联酋富查伊拉酋...,阿联酋修建巨型地下战略油库
1,2019-02-28 23:55:01,【海航辟谣破产传言：称最艰难时期已过 正全力化解流动性风险】针对近期网传“海航破产”传闻，海...,海航辟谣破产传言：称最艰难时期已过 正全力化解流动性风险
2,2019-02-28 23:48:31,【熊猫金控剥离银湖网生变 股东表决大会被取消】2月28日，熊猫金控股份有限公司(下称“熊猫金...,熊猫金控剥离银湖网生变 股东表决大会被取消
3,2019-02-28 23:37:32,【行情】东方财富网28日讯，在岸人民币兑美元(CNY)北京时间23:30收报6.6935元，...,在岸人民币兑美元(CNY)北京时间23:30收报6.6935元
4,2019-02-28 23:35:24,【美股三大股指窄幅震荡 京东财报靓丽股价涨逾5%】东方财富网28日讯，美东时间周四，美股三大...,美股三大股指窄幅震荡 京东财报靓丽股价涨逾5%
...,...,...,...
995,2019-02-26 14:13:08,【创业板涨近3%】东方财富网26日讯，创业板涨近3%，两市成交额破9000亿元。,创业板涨近3%
996,2019-02-26 14:10:16,【央行副行长潘功胜：扩大金融服务业双向开放】央行副行长、国家外汇管理局局长潘功胜在接受央视财...,扩大金融服务业双向开放
997,2019-02-26 14:07:48,【宁夏出台22条措施促进跨境贸易便利化】记者26日从宁夏回族自治区政府获悉，为优化口岸营商环...,宁夏出22条措施促进跨境贸易便利化
998,2019-02-26 14:07:22,【中国铁建再获莫斯科地铁工程合同】莫斯科地铁第三换乘环线东段盾构施工合同签约仪式25日在莫斯...,中铁建再获莫斯科地铁工程合同


In [3]:
from transformers import BertModel, BertTokenizer
import torch
import tensorflow as tf

In [7]:
tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm-ext")
model = BertModel.from_pretrained("hfl/chinese-bert-wwm-ext")

pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

'2.15.0'